## Preprocessing

In [1]:
# TODO: comment
import pandas as pd
import numpy as np
#
import warnings
warnings.filterwarnings("ignore")
#

In [2]:
# TODO: comment
path_to_file = "../data/Component_Faults_Data.csv"
df = pd.read_csv(path_to_file)

In [3]:
# TODO: comment
x = df.iloc[:, :48].values
y = df["class"].values.reshape(-1, 1)

In [4]:
# TODO: comment
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x = sc.fit_transform(x)

In [5]:
# TODO: comment
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

In [6]:
# TODO: comment
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

## Data Exploration

In [7]:
# TODO: stats

In [8]:
# TODO: plot

## Newer Solution for testing NN-Configuration

In [9]:
# Imports for class
import sys
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt    
import itertools
        

""" 
    The following class helps in streamlining the building, training, and testing of a Neural Network (NN)
    With it, we can see the effect on the result for changing a single parameter 
    So we only have to provide the name of the parameter we have to change, and the values we want to test it with
"""
class NeuralNetworkParameterTester:

    # The config holds all the changeable parameters for building, training and testing the NN
    config = None
    result = []  # Result of the test
    
    def __init__(self, set_default_config=True):
        
        if set_default_config:
            default_config = {
                'number_of_hidden_layers': [1],
                'number_of_units_per_hidden_layer': [10],
                'epochs': [100],
                'batch_size': [64],
                'activation_function': ['relu'],
                'loss_function': ['categorical_crossentropy'],
                'optimizer': ['sgd']}

            self.config = default_config
        
    def run(self):
        
        for config in self.__build_all_config_combinations():
            model = self.__build(config)
            trained_model, history = self.__train(config, model)
            accuracy = self.__test(trained_model)

            # save result
            self.result.append({'params': config.copy(), 
                                "result": {'model': trained_model, 
                                           'accuracy': accuracy}})
            
    def __build_all_config_combinations(self):

        raw_combinations = list(itertools.product(*(self.config[parameter] for parameter in self.config)))

        config_combinations_list = []
        for combination in raw_combinations:
            c = {
                'number_of_hidden_layers': combination[0],
                'number_of_units_per_hidden_layer': combination[1],
                'epochs': combination[2],
                'batch_size': combination[3],
                'activation_function': combination[4],
                'loss_function': combination[5],
                'optimizer': combination[6]}
            
            config_combinations_list.append(c)
        
        return config_combinations_list

    def __build(self, test_config):

        # Sequential model (Basic NN)
        model = Sequential()        
        # Building of input layer
        model.add(Dense(test_config['number_of_units_per_hidden_layer'], 
                        input_dim=48, 
                        activation=test_config['activation_function']))
        # Building of hidden layer(s)
        for i in range(test_config['number_of_hidden_layers']):
            model.add(Dense(test_config['number_of_units_per_hidden_layer'], 
                            activation=test_config['activation_function']))
        # Building of output layer
        model.add(Dense(11, activation="softmax"))
        # ?
        model.compile(loss=test_config['loss_function'], 
                      optimizer=test_config['optimizer'], 
                      metrics=['accuracy'])
        return model

    def __train(self, test_config, model):
        # xtrain and ytrain is the data from preprocessing
        history = model.fit(x_train, 
                            y_train, 
                            epochs=test_config['epochs'], 
                            batch_size=test_config['batch_size'])  
        
        return model, history
    
    
    # TODO: look at this method more closely. So far just copy paste
    @staticmethod
    def __test(model):
        y_pred = model.predict(x_test)
        # Converting predictions to label
        pred = list()
        for i in range(len(y_pred)):
            pred.append(np.argmax(y_pred[i]))
        # Converting one hot encoded test label to label
        test = list()
        for i in range(len(y_test)):
            test.append(np.argmax(y_test[i]))

        from sklearn.metrics import accuracy_score
        accuracy = accuracy_score(pred, test)
        return accuracy
                                    
                                    
    # Setter methods for setting single parameters
    def set_number_of_hidden_layers(self, val):
        if isinstance(val, list):
            self.config["number_of_hidden_layers"] = val
        else:
            self.config["number_of_hidden_layers"] = [val]
        
    def set_number_of_units_per_hidden_layer(self, val):
        if isinstance(val, list):
            self.config["number_of_units_per_hidden_layer"] = val
        else:
            self.config["number_of_units_per_hidden_layer"] = [val]
        
    def set_activation_function(self, val):
        if isinstance(val, list):
            self.config["activation_function"] = val
        else:
            self.config["activation_function"] = [val]
        
    def set_epochs(self, val):
        if isinstance(val, list):
            self.config["epochs"] = val
        else:
            self.config["epochs"] = [val]
        
    def set_batch_size(self, val):
        if isinstance(val, list):
            self.config["batch_size"] = val
        else:
            self.config["batch_size"] = [val]
        
    def set_loss_function(self, val):
        if isinstance(val, list):
            self.config["loss_function"] = val
        else:
            self.config["loss_function"] = [val]
        
    def set_optimizer(self, val):
        if isinstance(val, list):
            self.config["optimizer"] = val
        else:
            self.config["optimizer"] = [val]
        
    # This method is helpful if you just want to reset the changes you made to the parameters
    def reset_config(self):
        default_config = {
            'number_of_hidden_layers': [1],
            'number_of_units_per_hidden_layer': [10],
            'epochs': [100],
            'batch_size': [64],
            'activation_function': ['relu'],
            'loss_function': ['categorical_crossentropy'],
            'optimizer': ['sgd']}
        
        self.config = config
        
    def __get_number_of_params_with_multiple_vals(self):
        number_of_params_with_multiple_vals = 0 
        for key in self.config.keys():
            if isinstance(self.config[key], list):
                number_of_params_with_multiple_vals += 1
        return number_of_params_with_multiple_vals
        
    # This method plots the result without needing any input from the user
    def plot_result(self):
        if self.__get_number_of_params_with_multiple_vals() <= 1:
            self.__plot_2d()
        else:
            print("Not supported")

    def __plot_2d(self):

        # TODO: improve plot

        test_param_val = [element['test_param_val'] for element in self.result]
        accuracy_result = [element['result']['accuracy'] for element in self.result]

        plt.scatter(test_param_val, accuracy_result)
        plt.plot(test_param_val, accuracy_result, linestyle='--')
        plt.title("Accuracy per " + "'" + self.test_param_name + "'")
        plt.ylabel("Accuracy")
        plt.xlabel(self.test_param_name)
        plt.show()
        # TODO : plot as table
        print('"test_param_val":\t', test_param_val)
        print('"accuracy_result":\t', accuracy_result)

Using TensorFlow backend.


## Test NN with parameters

In [10]:
nnpt = NeuralNetworkParameterTester()

In [11]:
nnpt.set_number_of_hidden_layers([2, 5, 10])
nnpt.set_number_of_units_per_hidden_layer([2, 4])

In [12]:
nnpt.config

{'number_of_hidden_layers': [2, 5, 10],
 'number_of_units_per_hidden_layer': [2, 4],
 'epochs': [100],
 'batch_size': [64],
 'activation_function': ['relu'],
 'loss_function': ['categorical_crossentropy'],
 'optimizer': ['sgd']}

In [ ]:
nnpt.run()






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100





26329/26329 [==============================] - 1s 27us/step - loss: 2.4281 - acc: 0.0858
Epoch 2/100
26329/26329 [==============================] - 0s 14us/step - loss: 2.3980 - acc: 0.0885
Epoch 3/100
26329/26329 [==============================] - 0s 14us/step - loss: 2.3959 - acc: 0.0904
Epoch 4/100
26329/26329 [==============================] - 0s 14us/step - loss: 2.3939 - acc: 0.1001
Epoch 5/100
26329/26329 [==============================] - 0s 14us/step - loss: 2.3907 - acc: 0.1108
Epoch 6/100
26329/26329 [==============================] - 0s 14us/step - loss: 2.3839 - acc: 0.1139
Epoch 7/100
26329/26329 [==============================] - 0s 15us/step - loss: 2.3674 - acc: 0.1344
Epoch 8/100
26329/26329 [==============================] - 0s 14us/step - loss: 2.3273 - acc: 0.1678
Epoch 9/100
26329/26329 [==============================] - ETA: 0s - loss: 2.2343 - a

26329/26329 [==============================] - 0s 14us/step - loss: 1.4827 - acc: 0.3565
Epoch 54/100
26329/26329 [==============================] - 0s 14us/step - loss: 1.4800 - acc: 0.3552
Epoch 55/100
26329/26329 [==============================] - 0s 14us/step - loss: 1.4767 - acc: 0.3532
Epoch 56/100
26329/26329 [==============================] - 0s 14us/step - loss: 1.4749 - acc: 0.3589
Epoch 57/100
26329/26329 [==============================] - 0s 14us/step - loss: 1.4727 - acc: 0.3575
Epoch 58/100
26329/26329 [==============================] - 0s 15us/step - loss: 1.4697 - acc: 0.3558
Epoch 59/100
26329/26329 [==============================] - 0s 14us/step - loss: 1.4680 - acc: 0.3598
Epoch 60/100
26329/26329 [==============================] - 0s 14us/step - loss: 1.4655 - acc: 0.3615
Epoch 61/100
26329/26329 [==============================] - 0s 14us/step - loss: 1.4634 - acc: 0.3617
Epoch 62/100
26329/26329 [==============================] - 0s 15us/step - loss: 1.4609 - acc: 

26329/26329 [==============================] - 0s 14us/step - loss: 0.3453 - acc: 0.8404
Epoch 34/100
26329/26329 [==============================] - 0s 14us/step - loss: 0.3432 - acc: 0.8428
Epoch 35/100
26329/26329 [==============================] - 0s 14us/step - loss: 0.3390 - acc: 0.8433
Epoch 36/100
26329/26329 [==============================] - 0s 14us/step - loss: 0.3373 - acc: 0.8421
Epoch 37/100
26329/26329 [==============================] - 0s 14us/step - loss: 0.3364 - acc: 0.8420
Epoch 38/100
26329/26329 [==============================] - 0s 15us/step - loss: 0.3332 - acc: 0.8460
Epoch 39/100
26329/26329 [==============================] - 0s 14us/step - loss: 0.3304 - acc: 0.8452
Epoch 40/100
26329/26329 [==============================] - 0s 14us/step - loss: 0.3286 - acc: 0.8466
Epoch 41/100
26329/26329 [==============================] - 0s 14us/step - loss: 0.3273 - acc: 0.8459
Epoch 42/100
26329/26329 [==============================] - 0s 14us/step - loss: 0.3254 - acc: 

26329/26329 [==============================] - 0s 16us/step - loss: 1.7330 - acc: 0.2459
Epoch 14/100
26329/26329 [==============================] - 0s 16us/step - loss: 1.6953 - acc: 0.2674
Epoch 15/100
26329/26329 [==============================] - 0s 16us/step - loss: 1.6681 - acc: 0.2855
Epoch 16/100
26329/26329 [==============================] - 0s 17us/step - loss: 1.6464 - acc: 0.2935
Epoch 17/100
26329/26329 [==============================] - 0s 16us/step - loss: 1.6280 - acc: 0.3065
Epoch 18/100
26329/26329 [==============================] - 0s 16us/step - loss: 1.6128 - acc: 0.3120
Epoch 19/100
26329/26329 [==============================] - 0s 16us/step - loss: 1.6000 - acc: 0.3148
Epoch 20/100
26329/26329 [==============================] - 0s 16us/step - loss: 1.5893 - acc: 0.3195
Epoch 21/100
26329/26329 [==============================] - 0s 17us/step - loss: 1.5789 - acc: 0.3230
Epoch 22/100
26329/26329 [==============================] - ETA: 0s - loss: 1.5695 - acc: 0.324

26329/26329 [==============================] - 0s 16us/step - loss: 1.1869 - acc: 0.5160
Epoch 93/100
26329/26329 [==============================] - 0s 16us/step - loss: 1.1911 - acc: 0.5175
Epoch 94/100
26329/26329 [==============================] - 0s 16us/step - loss: 1.1821 - acc: 0.5143
Epoch 95/100
26329/26329 [==============================] - 0s 16us/step - loss: 1.1838 - acc: 0.5192
Epoch 96/100
26329/26329 [==============================] - 0s 16us/step - loss: 1.1840 - acc: 0.5111
Epoch 97/100
26329/26329 [==============================] - 0s 16us/step - loss: 1.1753 - acc: 0.5205
Epoch 98/100
26329/26329 [==============================] - 0s 16us/step - loss: 1.1745 - acc: 0.5108
Epoch 99/100
26329/26329 [==============================] - 0s 16us/step - loss: 1.1718 - acc: 0.5131
Epoch 100/100
26329/26329 [==============================] - 0s 16us/step - loss: 1.1682 - acc: 0.5147
Epoch 1/100
26329/26329 [==============================] - 1s 37us/step - loss: 2.3279 - acc: 

26329/26329 [==============================] - 0s 17us/step - loss: 0.2380 - acc: 0.9133
Epoch 73/100
26329/26329 [==============================] - ETA: 0s - loss: 0.2456 - acc: 0.908 - 0s 17us/step - loss: 0.2461 - acc: 0.9090
Epoch 74/100
26329/26329 [==============================] - 0s 17us/step - loss: 0.2434 - acc: 0.9093
Epoch 75/100
26329/26329 [==============================] - 0s 17us/step - loss: 0.2415 - acc: 0.9098
Epoch 76/100
26329/26329 [==============================] - 0s 17us/step - loss: 0.2418 - acc: 0.9111
Epoch 77/100
26329/26329 [==============================] - 0s 17us/step - loss: 0.2437 - acc: 0.9107
Epoch 78/100
26329/26329 [==============================] - 1s 20us/step - loss: 0.2408 - acc: 0.9093
Epoch 79/100
26329/26329 [==============================] - 1s 20us/step - loss: 0.2475 - acc: 0.9090
Epoch 80/100
26329/26329 [==============================] - 1s 21us/step - loss: 0.2399 - acc: 0.9128
Epoch 81/100
26329/26329 [==============================]

In [ ]:
nnpt.result

In [ ]:
nnpt.plot_result()

In [ ]:
# TODO: Plot Matrix